In [141]:
import torch
import torch.nn as nn
import torchvision
from torch.autograd import Variable
import torchvision.transforms as transforms

import pandas as pd
import numpy as np

test_size = 0.33
batch_size = 128

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

d_frame = pd.read_csv("data/casp_regression.csv", sep=",", index_col=None)

dataset = d_frame.values

test_dataset = dataset[0:int(test_size * len(dataset)),:]
train_dataset = dataset[int(test_size * len(dataset)):,:]

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [142]:
n_output_neuron = 1
n_input_neuron = X_train.shape[1]
hidden_size = 100
learning_rate = 0.0001

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out
    
model = NeuralNet(n_input_neuron, hidden_size, n_output_neuron).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

In [143]:
num_epochs = 50
total_step = len(train_loader)


for epoch in range(num_epochs):
    loss_acc = np.zeros(len(train_loader))
    for i, features in enumerate(train_loader):  

        target = features[:,0]
        feature = features[:,1:]
        
        # Move tensors to the configured device
        target = target.to(device).float()
        feature = feature.to(device).float()
        
        # Forward pass
        output = model(feature)
        
        # Compute loss
        loss = criterion(output, target)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_acc[i] = loss.item()
    print ('Epoch [{}], Loss: {:.4f}' .format(epoch+1, loss_acc.mean()))
    print()
            


Epoch [1], Loss: 1339985575.1014

Epoch [2], Loss: 25403.6074

Epoch [3], Loss: 25081.5091

Epoch [4], Loss: 24893.2751

Epoch [5], Loss: 24686.4199

Epoch [6], Loss: 24466.7420

Epoch [7], Loss: 24250.2418

Epoch [8], Loss: 25492.9914

Epoch [9], Loss: 23704.9459

Epoch [10], Loss: 23380.3649

Epoch [11], Loss: 23024.0129

Epoch [12], Loss: 22605.6961

Epoch [13], Loss: 22165.5537

Epoch [14], Loss: 21653.1519

Epoch [15], Loss: 21099.4344

Epoch [16], Loss: 20549.0253

Epoch [17], Loss: 19905.3921

Epoch [18], Loss: 19181.5619

Epoch [19], Loss: 18403.4196

Epoch [20], Loss: 17563.9120

Epoch [21], Loss: 16693.7697

Epoch [22], Loss: 16667.0199

Epoch [23], Loss: 14852.5633

Epoch [24], Loss: 13704.1668

Epoch [25], Loss: 12613.2351

Epoch [26], Loss: 11575.3006

Epoch [27], Loss: 10512.0314

Epoch [28], Loss: 9380.6678

Epoch [29], Loss: 8327.6774

Epoch [30], Loss: 7241.8765

Epoch [31], Loss: 6305.9178

Epoch [32], Loss: 5379.6132

Epoch [33], Loss: 4540.2229

Epoch [34], Loss: 38

In [144]:
# test the trained model
ms_error = list()

with torch.no_grad():
    for test_item in test_loader:
        feature = test_item[:,1:]
        target = test_item[:,0]
        
        target = target.to(device).float()
        feature = feature.to(device).float()
        
        output = model.forward(feature)
        output = torch.reshape(output, (output.size(0),))

        ms_error.append(((output-target)**2).mean())

print("Mean squared error: %.4f" % np.mean(ms_error))

Mean squared error: 2031.7212
